# Julia Fundamentals

This chapter provides a comprehensive introduction to the Julia programming language, covering everything you need to work effectively with Durbyn.jl. Julia combines the expressiveness of Python with the performance of C, making it an ideal language for scientific computing and time series analysis.

## Why Julia for Time Series?

Julia was designed from the ground up for numerical and scientific computing:

- **Performance**: Just-in-time (JIT) compilation produces machine-native code. No "two-language problem" — prototyping code *is* production code.
- **Multiple Dispatch**: Julia's type system and dispatch mechanism enable elegant, extensible APIs (the foundation of Durbyn's formula grammar).
- **Composability**: Packages interoperate naturally through shared abstract types and interfaces (e.g., Tables.jl).
- **Built-in Parallelism**: Native threading support enables Durbyn to fit models across thousands of series concurrently.

## Variables and Basic Types

Julia is dynamically typed but leverages type inference for performance.

### Numeric Types

In [1]:
# Integers
x = 42           # Int64 (default on 64-bit systems)
println(typeof(x))         # Int64

# Floating-point
y = 3.14          # Float64
println(typeof(y))       # Float64

# Explicit types
z = Float32(3.14)
w = Int32(42)

# Arithmetic
a = 10 + 3        # 13
b = 10 / 3        # 3.333... (always Float64)
c = 10 ÷ 3        # 3 (integer division, type \div + Tab)
d = 10 % 3        # 1 (modulo operator, which calculates the remainder of a division)
e = 2^10          # 1024 (exponentiation)

Int64
Float64


1024


### Boolean and String Types

In [2]:
# Booleans
flag = true
typeof(flag)      # Bool

# Logical operators
!true             # false
true && false     # false
true || false     # true

# Strings (double quotes only)
name = "Durbyn"
typeof(name)      # String

# Characters (single quotes)
ch = 'D'
typeof(ch)        # Char

# String interpolation
version = 0.1
msg = "Package: $name v$(version)"
println("msg: ", msg)

# Multi-line strings
description = """
Durbyn.jl is a Julia package
for time series forecasting.
"""
println(description)

# String operations
length(name)          # 6
uppercase(name)       # "DURBYN"
occursin("byn", name) # true
split("a,b,c", ",")  # ["a", "b", "c"]
join(["a", "b"], "-") # "a-b"

msg: Package: Durbyn v0.1
Durbyn.jl is a Julia package
for time series forecasting.



"a-b"


### Special Values

In [3]:
# Nothing (absence of value)
x = nothing
typeof(x)         # Nothing
isnothing(x)      # true

# Missing (statistical missing data)
y = missing
typeof(y)         # Missing
ismissing(y)      # true

# NaN (not a number, for floating-point)
z = NaN
isnan(z)          # true

# Infinity
Inf               # positive infinity
-Inf              # negative infinity
isinf(Inf)        # true

true

> ❗ **Important**: `missing` vs `nothing` vs `NaN`
>
>
> - **`nothing`** — the absence of a value (like `None` in Python, `NULL` in R). Used for optional parameters.
> - **`missing`** — a *statistical* missing value that propagates through operations: `1 + missing` returns `missing`.
> - **`NaN`** — a floating-point sentinel for undefined numerical results: `0/0` returns `NaN`.
>
> In Durbyn.jl, time series data may contain `NaN` for missing observations. Use `interpolate_missing`, `longest_contiguous`, or `check_missing` to handle them. The `handle_missing` function dispatches to the appropriate strategy via type dispatch (e.g., `handle_missing(y, Contiguous())`).


## Collections

### Arrays (Vectors and Matrices)

Arrays are the fundamental data structure for time series in Durbyn and there are other data types for time series in Julia but we did not use due to efficiency.

In [4]:
# 1-D Array (Vector)
v = [1, 2, 3, 4, 5]
typeof(v)             # Vector{Int64} (alias for Array{Int64, 1})

# Float vector
vf = [1.0, 2.0, 3.0]

# Empty typed vector
empty_v = Float64[]

# Range objects (lazy — no allocation)
r = 1:10              # 1, 2, ..., 10
r2 = 0.0:0.1:1.0     # 0.0, 0.1, 0.2, ..., 1.0
collect(r)            # materialise into a Vector

# Indexing (1-based!)
v[1]                  # 1 (first element)
v[end]                # 5 (last element)
v[2:4]                # [2, 3, 4] (slice)
v[end-1:end]          # [4, 5]

# Constructors
zeros(5)              # [0.0, 0.0, 0.0, 0.0, 0.0]
ones(3)               # [1.0, 1.0, 1.0]
fill(7, 4)            # [7, 7, 7, 7]
rand(5)               # 5 uniform random numbers in [0, 1)
randn(5)              # 5 standard normal random numbers

# 2-D Array (Matrix)
M = [1 2 3; 4 5 6]   # 2×3 matrix
size(M)               # (2, 3)
M[1, 2]               # 2 (row 1, column 2)
M[:, 1]               # [1, 4] (first column)
M[2, :]               # [4, 5, 6] (second row)

3-element Vector{Int64}:
 4
 5
 6


### Element-wise Operations (Broadcasting)

The dot syntax `.` broadcasts operations element-wise — essential for time series transformations.

In [5]:
x = [1.0, 2.0, 3.0, 4.0]

# Element-wise operations
x .+ 10              # [11.0, 12.0, 13.0, 14.0]
x .* 2               # [2.0, 4.0, 6.0, 8.0]
x .^ 2               # [1.0, 4.0, 9.0, 16.0]
log.(x)              # [0.0, 0.693, 1.099, 1.386]
sqrt.(x)             # element-wise square root

# Comparison broadcasting
x .> 2               # [false, false, true, true]

# The @. macro broadcasts everything
@. log(x + 1) / x   # equivalent to log.(x .+ 1) ./ x

4-element Vector{Float64}:
 0.6931471805599453
 0.5493061443340549
 0.46209812037329684
 0.40235947810852507


### Tuples and Named Tuples

Named tuples are the backbone of Durbyn's table system.

In [6]:
# Tuple (immutable, ordered)
t = (1, "hello", 3.14)
t[1]                  # 1

# Named Tuple (column-oriented table)
tbl = (
    date  = [2020, 2021, 2022],
    value = [100.0, 120.0, 135.0],
    label = ["a", "b", "c"]
)
tbl.date              # [2020, 2021, 2022]
tbl.value[2]          # 120.0

# This is exactly the format Durbyn.TableOps expects!

120.0


### Dictionaries

In [ ]:
# Create a dictionary
params = Dict(
    "alpha" => 0.3,
    "beta"  => 0.1,
    "gamma" => 0.05
)

# Access
params["alpha"]       # 0.3
get(params, "phi", 0.98)  # 0.98 (default if key missing)
haskey(params, "beta")    # true

# Iterate
for (k, v) in params
    println("$k = $v")
end

# Keys and values
keys(params)
values(params)

alpha = 0.3
gamma = 0.05
beta = 0.1


ValueIterator for a Dict{String, Float64} with 3 entries. Values:
  0.3
  0.05
  0.1


## Control Flow

### Conditionals

In [8]:
x = 42

# if-elseif-else
if x > 100
    println("large")
elseif x > 10
    println("medium")
else
    println("small")
end

# Ternary operator
label = x > 50 ? "high" : "low"

# Short-circuit evaluation
x > 0 && println("positive")    # prints if true
x < 0 || println("non-negative") # prints if first is false

medium
positive
non-negative



### Loops

In [ ]:
# for loop with range
for i in 1:5
    println(i)
end

# Iterating over collections
names = ["ARIMA", "ETS", "Theta"]

for name in names
    println(name)
end

# Enumerate (index + value)
for (i, name) in enumerate(names)
    println("$i: $name")
end

# while loop
n = 1
while n <= 10
    n *= 2
end

# Comprehensions (concise array construction)
squares = [x^2 for x in 1:10]
evens = [x for x in 1:20 if x % 2 == 0]

# Generator expressions (lazy — no allocation)
total = sum(x^2 for x in 1:1000)

1
2
3
4
5
ARIMA
ETS
Theta
1: ARIMA
2: ETS
3: Theta


333833500


## Functions

Functions are first-class objects in Julia. The language's performance relies on specialising functions by argument types.

### Defining Functions

In [27]:
# Standard form
function greet(name)
    return "Hello, $(name)!"
end

# Short form (for one-liners)
greet(name) = "Hello, $(name)!"

# Call
greet("TAFS")  # "Hello, TAFS!"

"Hello, TAFS!"


### Arguments and Return Values

In [ ]:
# Positional and keyword arguments
function forecast_horizon(n; h=1, seasonal=false)
    base = seasonal ? n * 2 : n
    return base + h
end

forecast_horizon(100)               # 101
forecast_horizon(100, h=12)         # 112
forecast_horizon(100, h=12, seasonal=true) # 212

# Default values
function ets_model(y, m; model="ZZZ", damped=nothing)
    println("Fitting ETS($model) with m=$m, damped=$damped")
end

# Multiple return values
function minmax(x)
    return minimum(x), maximum(x)
end

lo, hi = minmax([3, 1, 4, 1, 5])   # lo=1, hi=5

# Mutating convention: functions ending in ! modify their arguments
x = [3, 1, 4]
sort(x)    # returns [1, 3, 4], x unchanged
sort!(x)   # modifies x in-place

3-element Vector{Int64}:
 1
 3
 4


### Anonymous Functions (Lambdas)

Anonymous functions are used extensively in Durbyn's `query`, `mutate`, and `summarise`.

In [29]:
# Lambda syntax
square = x -> x^2
square(5)             # 25

# Multi-argument
add = (x, y) -> x + y

# do-block syntax (for longer anonymous functions)
result = map([1, 2, 3]) do x
    x^2 + 1
end
# [2, 5, 10]

# Used in Durbyn.TableOps:
# query(tbl, row -> row.value > 100)
# mutate(tbl, :log_value => row -> log(row.value))

3-element Vector{Int64}:
  2
  5
 10


### Higher-Order Functions

In [3]:
data = [1, 2, 3, 4, 5]

# map: apply function to each element
map(x -> x^2, data)          # [1, 4, 9, 16, 25]

# filter: keep elements satisfying predicate
filter(x -> x > 3, data)     # [4, 5]

# reduce: fold elements together
println(reduce(+, data))              # 15

# Broadcasting is often preferred over map:
data .^ 2                    # [1, 4, 9, 16, 25]

15


5-element Vector{Int64}:
  1
  4
  9
 16
 25


## Type System and Multiple Dispatch

Julia's type system and multiple dispatch are central to how Durbyn.jl provides a unified `fit`/`forecast`/`accuracy` API across different model types.

### Abstract and Concrete Types

In [31]:
# Abstract type (cannot be instantiated)
abstract type AbstractModel end

# Concrete types (can be instantiated)
struct ETSModel <: AbstractModel
    method::String
    alpha::Float64
    beta::Float64
end

struct ARIMAModel <: AbstractModel
    order::Tuple{Int, Int, Int}
    coefficients::Vector{Float64}
end

# Type hierarchy
ETSModel <: AbstractModel     # true
ARIMAModel <: AbstractModel   # true

true


### Multiple Dispatch

The same function name dispatches to different methods based on argument types:

In [ ]:
# Generic function with multiple methods
function describe(m::ETSModel)
    println("ETS($(m.method)): α=$(m.alpha), β=$(m.beta)")
end

function describe(m::ARIMAModel)
    p, d, q = m.order
    println("ARIMA($p,$d,$q) with $(length(m.coefficients)) coefficients")
end

# Julia selects the correct method at runtime
ets = ETSModel("AAN", 0.3, 0.1)
arima = ARIMAModel((2, 1, 1), [0.5, -0.3, 0.2])

describe(ets)    # calls ETSModel method
describe(arima)  # calls ARIMAModel method

ETS(AAN): α=0.3, β=0.1
ARIMA(2,1,1) with 3 coefficients


> 📝 **Note**: How Durbyn Uses Multiple Dispatch
>
>
> Durbyn.jl defines `fit`, `forecast`, `accuracy`, `plot`, `fitted`, `residuals`, and `summary` as generic functions with specialised methods for each model spec (`ArimaSpec`, `EtsSpec`, `BatsSpec`, etc.). This means you use the same API regardless of the model:
>

In [33]:
fc = forecast(fit(spec, data, m=12), h=12)

UndefVarError: UndefVarError: `fit` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

>
> The `spec` type determines which fitting algorithm runs.


### Parametric Types

In [34]:
# Parametric struct
struct TimeSeries{T <: Real}
    values::Vector{T}
    frequency::Int
end

# Works with any numeric type
ts_float = TimeSeries([1.0, 2.0, 3.0], 12)
ts_int   = TimeSeries([1, 2, 3], 4)

TimeSeries{Int64}([1, 2, 3], 4)


## Modules and Packages

### Using Packages

In [ ]:
# Import everything exported by a package
using Durbyn

# Import specific names
using Durbyn: fit, forecast, accuracy

# Import a submodule
using Durbyn.TableOps
using Durbyn.ExponentialSmoothing

# Qualified access (no import into namespace)
import Durbyn
Durbyn.fit(spec, data)


### Package Management

In [ ]:
using Pkg

# Add packages
Pkg.add("CSV")
Pkg.add(url="https://github.com/taf-society/Durbyn.jl")

# Update packages
Pkg.update()

# Check status
Pkg.status()

# Project environments (recommended)
# Create a new environment in current directory:
Pkg.activate(".")
Pkg.add("Durbyn")

> 💡 **Tip**: Environment Best Practice
>
>
> Always work within a project environment. This ensures reproducibility:
>

In [2]:
using Pkg
Pkg.activate("workshop");
Pkg.add(url="https://github.com/taf-society/Durbyn.jl");
Pkg.add(["Durbyn", "CSV", "Plots"]);

  Activating project at `~/Development/Personal/WorkShops/DL4SG-Training-Forecasting-in-Julia-using-Durbyn.jl/notebooks/workshop`
    Updating git-repo `https://github.com/taf-society/Durbyn.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/Development/Personal/WorkShops/DL4SG-Training-Forecasting-in-Julia-using-Durbyn.jl/notebooks/workshop/Project.toml`
  [40b042de] ~ Durbyn v0.0.1 `https://github.com/taf-society/Durbyn.jl#main` ⇒ v0.0.1 `https://github.com/taf-society/Durbyn.jl#main`
    Updating `~/Development/Personal/WorkShops/DL4SG-Training-Forecasting-in-Julia-using-Durbyn.jl/notebooks/workshop/Manifest.toml`
  [40b042de] ~ Durbyn v0.0.1 `https://github.com/taf-society/Durbyn.jl#main` ⇒ v0.0.1 `https://github.com/taf-society/Durbyn.jl#main`
Precompiling packages...
   8323.6 ms  ✓ Durbyn
  1 dependency successfully precompiled in 9 seconds. 56 already precompiled.
   Resolving package versions...


Pkg.Types.PkgError: expected package `Durbyn [40b042de]` to be registered

>
> The `Project.toml` and `Manifest.toml` files record exact package versions.


## Common Patterns for Time Series Work

### Working with Missing Data

In [37]:
using Statistics
# Missing propagates
1 + missing              # missing
[1, missing, 3] .+ 1    # [2, missing, 4]

# skipmissing: iterate over non-missing values
data = [1.0, missing, 3.0, missing, 5.0]
mean(skipmissing(data))  # 3.0
collect(skipmissing(data))  # [1.0, 3.0, 5.0]

# coalesce: replace missing with a default
coalesce(missing, 0.0)   # 0.0
coalesce.(data, 0.0)     # [1.0, 0.0, 3.0, 0.0, 5.0]

# Filter rows with missing values (Durbyn style)
# query(tbl, row -> coalesce(row.value > 0, false))

5-element Vector{Float64}:
 1.0
 0.0
 3.0
 0.0
 5.0


### Pipeline Style with `|>`

In [38]:
# The pipe operator passes the left side as the first argument
result = [3, 1, 4, 1, 5, 9] |> sort |> unique |> reverse
# [9, 5, 4, 3, 1]

# Useful for chaining operations on data
data = rand(100)
data |> mean
data |> x -> filter(v -> v > 0.5, x) |> length

61


### Working with Dates

In [42]:
using Dates

# Create dates
d = Date(2026, 2, 19)
dt = DateTime(2026, 2, 19, 10, 30, 0)

# Date arithmetic
d + Day(30)
d + Month(3)

# Date ranges
dates = Date(2020, 1):Month(1):Date(2022, 12)
collect(dates)            # monthly dates from Jan 2020 to Dec 2022

# Parsing
Date("2024-01-15", dateformat"yyyy-mm-dd")
Date("15/01/2024", dateformat"dd/mm/yyyy")

# Components
year(d)
month(d)
dayofweek(d)

4